In [4]:
import torch
import pandas as pd
import polars as pl
import numpy as np
import requests
import time
import json
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support, classification_report
from tqdm import tqdm

# Configuration
MAX_ORES_QUERIES = 20000  # None for FULL TEST but probably overkill?
OUTPUT_FILE = "ores_benchmark_results.json"

def get_test_set_articles():
    """Get same test set articles with revision IDs"""

    # Load data and create sam test mask as training
    data = torch.load("data/data_quantile_Target_QC_aggcat.pt", weights_only=False)

    # Import mask creation function
    from utils.create_split_masks import create_split_masks
    _, _, test_mask = create_split_masks(data, seed=42)
    test_indices = torch.where(test_mask)[0].cpu().numpy()

    print(f"Test set size: {len(test_indices)} nodes")

    # Get true labels for test set
    test_labels = data.y[test_indices].cpu().numpy()

    # Load wiki data to get pageids
    df_wiki = pl.read_parquet("data/df_wiki_data.parquet")
    test_pageids = df_wiki['pageid'][test_indices].to_list()

    # Load original articles to get revision IDs
    df_original = pl.read_csv("data/cleaned_articles_final.csv")

    test_articles = df_original.filter(
        pl.col('pageid').is_in(test_pageids)
    ).select(['pageid', 'article', 'quality_class', 'revid'])

    # Add node index and true label
    pageid_to_idx = {pid: idx for idx, pid in enumerate(df_wiki['pageid'].to_list())}
    pageid_to_label = {test_pageids[i]: test_labels[i] for i in range(len(test_pageids))}

    test_articles = test_articles.with_columns([
        pl.col('pageid').map_elements(
            lambda x: pageid_to_idx.get(x),
            return_dtype=pl.Int64
        ).alias('node_index'),
        pl.col('pageid').map_elements(
            lambda x: pageid_to_label.get(x),
            return_dtype=pl.Int64
        ).alias('true_label_numeric')
    ])

    # Filter to only test indices
    test_articles = test_articles.filter(
        pl.col('node_index').is_in(test_indices.tolist())
    )

    return test_articles.to_pandas()

def query_ores_api(revid):
    """Query ORES API with revision idD"""

    url = "https://api.wikimedia.org/service/lw/inference/v1/models/enwiki-articlequality:predict"
    headers = {
        "User-Agent": "UniversityofKonstanzArticleResearchBot/1.0",
        "Content-Type": "application/json",
    }

    try:
        resp = requests.post(
            url,
            headers=headers,
            json={"rev_id": int(revid)},
            timeout=15
        )
        resp.raise_for_status()
        j = resp.json()

        scores = j.get("enwiki", {}).get("scores", {})
        rev_key = next(iter(scores.keys()), None)

        if rev_key:
            pred = scores[rev_key].get("articlequality", {}).get("score", {}).get("prediction")
            probs = scores[rev_key].get("articlequality", {}).get("score", {}).get("probability", {})
            return pred, probs

    except Exception as e:
        print(f"Error for revid {revid}: {e}")

    return None, None

# Get test articles
test_articles_df = get_test_set_articles()
print(f"Found {len(test_articles_df)} articles")

# Limit for testing if needed
if MAX_ORES_QUERIES:
    test_articles_df = test_articles_df.sample(n=min(MAX_ORES_QUERIES, len(test_articles_df)), random_state=42)

# Launch attack on ORES.API
print("\nOK GARMIN QUERY THE ORES API WITH OUR TEST DATA...")
ores_predictions = []
ores_probabilities = []

for idx, row in tqdm(test_articles_df.iterrows(), total=len(test_articles_df)):
    pred, probs = query_ores_api(row['revid'])
    ores_predictions.append(pred)
    ores_probabilities.append(probs)
    time.sleep(0.2)  # Rate limit ᶻ 𝗓 𐰁(ᴗ_ ᴗ。)ᶻ 𝗓 𐰁(ᴗ_ ᴗ。)ᶻ 𝗓 𐰁(ᴗ_ ᴗ。)

test_articles_df['ores_prediction'] = ores_predictions
test_articles_df['ores_probabilities'] = ores_probabilities

# Map to numeric
quality_to_num = {
    "FA": 9, "FL": 8, "FM": 7, "A": 6,
    "GA": 5, "B": 4, "C": 3, "Start": 2,
    "Stub": 1, "List": 0
}

# Aggregated categories
quality_to_aggcat = {
    "FA": 2, "FL": 2, "FM": 2,
    "A": 1, "GA": 1, "B": 1, "C": 1,
    "Start": 0, "Stub": 0, "List": 0
}

# Map to aggregated categories
test_articles_df['actual_aggcat'] = test_articles_df['quality_class'].map(quality_to_aggcat)
test_articles_df['ores_aggcat'] = test_articles_df['ores_prediction'].map(quality_to_aggcat)

# Filter valids
valid_df = test_articles_df.dropna(subset=['ores_aggcat'])
print(f"\nValid ORES predictions: {len(valid_df)}/{len(test_articles_df)}")

# Metrics
y_true = valid_df['true_label_numeric'].values
y_pred = valid_df['ores_aggcat'].values

accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, average=None, zero_division=0)
precision_macro = precision.mean()
recall_macro = recall.mean()
f1_macro = f1.mean()
f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0)

# Results
print(f"Accuracy:{accuracy:.4f}")
print(f"Precision (macro):{precision_macro:.4f}")
print(f"Recall (macro):{recall_macro:.4f}")
print(f"F1 (macro):{f1_macro:.4f}")
print(f"F1 (weighted):{f1_weighted:.4f}")
print(classification_report(y_true, y_pred, target_names=['Low', 'Medium', 'High']))

# Save to JSON
results = {
    "model": "ORES",
    "test_size": int(len(valid_df)),
    "total_test_size": int(len(test_articles_df)),
    "failed_predictions": int(len(test_articles_df) - len(valid_df)),
    "metrics": {
        "accuracy": float(accuracy),
        "precision_macro": float(precision_macro),
        "recall_macro": float(recall_macro),
        "f1_macro": float(f1_macro),
        "f1_weighted": float(f1_weighted),
        "precision_per_class": [float(x) for x in precision.tolist()],
        "recall_per_class": [float(x) for x in recall.tolist()],
        "f1_per_class": [float(x) for x in f1.tolist()],
        "support_per_class": [int(x) for x in support.tolist()]
    },
    "class_distribution": {
        "true": {str(k): int(v) for k, v in zip(*np.unique(y_true, return_counts=True))},
        "predicted": {str(k): int(v) for k, v in zip(*np.unique(y_pred, return_counts=True))}
    }
}

with open(OUTPUT_FILE, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\nResults saved: {OUTPUT_FILE}")

# Save as csv
test_articles_df.to_csv("ores_detailed_predictions.csv", index=False)

Test set size: 56980 nodes
Found 56978 articles

OK GARMIN QUERY THE ORES API WITH OUR TEST DATA...


 15%|█▌        | 3021/20000 [53:06<4:36:16,  1.02it/s] 

Error for revid 1274708269: HTTPSConnectionPool(host='api.wikimedia.org', port=443): Read timed out. (read timeout=15)


 15%|█▌        | 3084/20000 [54:28<5:31:03,  1.17s/it] 

Error for revid 1297261759: HTTPSConnectionPool(host='api.wikimedia.org', port=443): Read timed out. (read timeout=15)


...

Error for revid 1301459657: HTTPSConnectionPool(host='api.wikimedia.org', port=443): Read timed out. (read timeout=15)


100%|█████████▉| 19931/20000 [11:18:45<01:10,  1.02s/it]   

Error for revid 1290452120: HTTPSConnectionPool(host='api.wikimedia.org', port=443): Read timed out. (read timeout=15)


100%|██████████| 20000/20000 [11:35:34<00:00,  2.09s/it]   



Valid ORES predictions: 19942/20000
Accuracy:0.6412
Precision (macro):0.5057
Recall (macro):0.6805
F1 (macro):0.4885
F1 (weighted):0.6663
              precision    recall  f1-score   support

         Low       0.97      0.55      0.70     14138
      Medium       0.44      0.86      0.58      5681
        High       0.10      0.63      0.18       123

    accuracy                           0.64     19942
   macro avg       0.51      0.68      0.49     19942
weighted avg       0.82      0.64      0.67     19942


Results saved: ores_benchmark_results.json
